In [1]:
!apt-get update
!apt-get install ffmpeg libsm6 libxext6  -y

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Reading package lists... Done
Reading package lists... Done
Building dependency t

In [2]:
!pip install seaborn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cp -av "/content/drive/MyDrive/chemp-ufa/train_dataset_train.zip" "./"

'/content/drive/MyDrive/chemp-ufa/train_dataset_train.zip' -> './train_dataset_train.zip'


In [5]:
!unzip ./train_dataset_train.zip

Archive:  ./train_dataset_train.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: train.csv               
  inflating: Уфа_baseline.ipynb   
  inflating: train/220301070305_0e13309ae71ffc37ba629d19f46e0784.jpg  
  inflating: train/220301070430_df1a737a5abe3707b424fbe9d1d92300.jpg  
  inflating: train/220301070439_91576ea7fbb6567ff743e271662f9d06.jpg  
  inflating: train/220301070525_8e9cf12c82b056104cfd8868f5958c3e.jpg  
  inflating: train/220301070535_78bd475d54865dc7b1c356f3436def6e.jpg  
  inflating: train/220301070614_4c1a339dd5056cd9747f3b4693f27b1b.jpg  
  inflating: train/220301070625_1be30d27fcfc119ed8dee6037a9ab385.jpg  
  inflating: train/220301070852_6431af3f1d3517498811dfa67f8a30be.jpg  
  inflating: train/220301070914_9f40eac64f72c8d6ee0e7f4fa37dc4ba.jpg  
  inflating: train/220301071234_2e7ed5acd0a8e8682b28f6618e7694c9.jpg  
  inflating: train/220301071302_180763d363e88725cc11143fb6923c1c.jpg  
  inflating: train/220301071500_fc5d48ef20ecb259c53

In [6]:
% mkdir ./my_train0-1-2
% mkdir ./my_train0-1-2/0
% mkdir ./my_train0-1-2/1
% mkdir ./my_train0-1-2/2

mkdir: cannot create directory ‘./my_train0-1-2’: File exists
mkdir: cannot create directory ‘./my_train0-1-2/0’: File exists
mkdir: cannot create directory ‘./my_train0-1-2/1’: File exists
mkdir: cannot create directory ‘./my_train0-1-2/2’: File exists


In [7]:
import pandas as pd 
import shutil

data_df = pd.read_csv("./train.csv")
for i in data_df.index:
    if data_df['class'].iloc[i]==0.0:
        shutil.copy('./train/'+data_df['ID_img'].iloc[i]+'', './my_train0-1-2/0/')

        
    if data_df['class'].iloc[i]==1.0:
        shutil.copy('./train/'+data_df['ID_img'].iloc[i]+'', './my_train0-1-2/1/')

        
    if data_df['class'].iloc[i]==2.0:
        shutil.copy('./train/'+data_df['ID_img'].iloc[i]+'', './my_train0-1-2/2/')

In [8]:
import pandas as pd 
import numpy as np
import glob
from tqdm import tqdm
import cv2

from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [9]:
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
#https://keras.io/api/applications/

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model, Sequential
import numpy as np
import pandas as pd
import shutil
import time
import cv2 as cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os
import seaborn as sns
sns.set_style('darkgrid')
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def generators(shape): 
    imgdatagen = ImageDataGenerator(
        rotation_range=0.2,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.5,
        horizontal_flip = True, 
        fill_mode='nearest',
        validation_split = 0.1,
    )

    height, width = shape

    train_dataset = imgdatagen.flow_from_directory(
        './my_train0-1-2/',
        target_size = (height, width), 
        batch_size = batch_size,
        class_mode='categorical',
        subset = 'training', 
    )

    val_dataset = imgdatagen.flow_from_directory(
        './my_train0-1-2/',
        target_size = (height, width), 
        batch_size = batch_size,
        class_mode='categorical',
        subset = 'validation'
    )
    return train_dataset, val_dataset

In [12]:
batch_size =3

In [13]:
model_name='EfficientNetB7'
base_model=tf.keras.applications.EfficientNetB7(include_top=False,input_shape=(600,600,3), weights="imagenet", pooling='max') 
x=base_model.output
x=keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
x = Dense(256, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
x=Dropout(rate=.45, seed=123)(x)        
output=Dense(3, activation='softmax')(x)
model=Model(inputs=base_model.input, outputs=output)
model.compile(Adamax(), loss='categorical_crossentropy', metrics=['accuracy']) 


In [14]:
train_dataset, val_dataset = generators((600,600))

Found 490 images belonging to 3 classes.
Found 53 images belonging to 3 classes.


In [15]:
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau


earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/content/drive/MyDrive/chemp-ufa/'+'pre_EfficientnetB7_{epoch:02d}-{val_loss:.2f}.hdf5', save_best_only=True)
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, min_delta=1e-4, mode='min')


In [16]:
print(tf.__version__)

2.8.2


In [17]:
#model = tf.keras.models.load_model('./'+'EfficientnetB7_32-0.96.hdf5')
#model.summary()

In [18]:

history = model.fit(
    train_dataset, 
    validation_data = val_dataset,
    epochs=150,
    callbacks=[earlyStopping, mcp_save, reduce_lr_loss]
    )

Epoch 1/50
164/164 [==============================] - 221s 1s/step - loss: 9.4536 - accuracy: 0.5571 - val_loss: 9.1819 - val_accuracy: 0.5849 - lr: 0.0010
Epoch 2/50
164/164 [==============================] - 184s 1s/step - loss: 8.2659 - accuracy: 0.6469 - val_loss: 8.0542 - val_accuracy: 0.7358 - lr: 0.0010
Epoch 3/50
164/164 [==============================] - 178s 1s/step - loss: 7.4526 - accuracy: 0.6408 - val_loss: 8.0745 - val_accuracy: 0.5849 - lr: 0.0010
Epoch 4/50
164/164 [==============================] - 184s 1s/step - loss: 6.7133 - accuracy: 0.6694 - val_loss: 7.1043 - val_accuracy: 0.7925 - lr: 0.0010
Epoch 5/50
164/164 [==============================] - 185s 1s/step - loss: 6.1932 - accuracy: 0.6571 - val_loss: 6.4657 - val_accuracy: 0.7170 - lr: 0.0010
Epoch 6/50
164/164 [==============================] - 185s 1s/step - loss: 5.6512 - accuracy: 0.6592 - val_loss: 5.6707 - val_accuracy: 0.7547 - lr: 0.0010
Epoch 7/50
164/164 [==============================] - 185s 1s/st